In [1]:

'''

28个被试 每个被试都有connectivity

'''

import os.path as op
import numpy as np
import mne
from mne_connectivity import spectral_connectivity_epochs
from mne.datasets import sample
from mne_connectivity.viz import plot_sensors_connectivity


In [ ]:

data_path = 'E:\Python LYW\RSA\spaced_learning\pre10\\analysis\data\\6epoch_clean_equal\\'

list_O_ids = [
    'prex006','prex009','prex010','prex011','prex012',
    'prex015','prex016','prex017','prex018','prex019','prex022','prex023',
    'prex027','prex028',
    'prex033','prex037','prex038','prex040',
    'prex041','prex042','prex044','prex045',
    'prex046','prex047','prex049','prex050','prex052','prex051'
]
#去掉了 5和9被
list_M_ids = [
    'prex006M','prex009M','prex010M','prex011M','prex012M',
    'prex015M','prex016M','prex017M','prex018M','prex019M','prex022M','prex023M',
    'prex027M','prex028M',
    'prex033M','prex037M','prex038M','prex040S',
    'prex041M','prex042M','prex044M','prex045M'
    ,'prex046M','prex047M','prex049S','prex050M','prex052M','prex051M'

]

list_S_ids = [
    'prex006S','prex009S','prex010S','prex011S','prex012S',
    'prex015S','prex016S','prex017S','prex018S','prex019S','prex022S','prex023S',
    'prex027S','prex028S'
    ,'prex033S','prex037S','prex038S','prex040M'
    ,'prex041S','prex042S','prex044S','prex045S'
    ,'prex046S','prex047S','prex049M','prex050S','prex052S','prex051S'
]


list_epochs_O = list()
list_epochs_S = list()
list_epochs_M = list()


for sub_id_O, sub_id_M, sub_id_S in zip(list_O_ids, list_M_ids, list_S_ids):
    epochs_O = mne.read_epochs(data_path+sub_id_O+'-epo.fif')
    epochs_M = mne.read_epochs(data_path+sub_id_M+'-epo.fif')
    epochs_S = mne.read_epochs(data_path+sub_id_S+'-epo.fif')
    list_epochs_O.append(epochs_O)
    list_epochs_M.append(epochs_M)
    list_epochs_S.append(epochs_S)


In [ ]:
# 频域范围选择
fmin, fmax = 4, 9
sfreq = 1000  # the sampling frequency
tmin = 0.25
tmax = 0.5 # exclude the baseline period
list_con_M = []
list_con_O = []
list_con_S = []
# con_avg_M = spectral_connectivity_epochs(
#         list_epochs_M[0], method='pli', mode='multitaper', sfreq=sfreq, fmin=fmin, fmax=fmax,
#         faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)
# con_avg_O = con_avg_M.copy()
# con_avg_S = con_avg_M.copy()


for i in range(len(list_epochs_S)):
    con = spectral_connectivity_epochs(
        list_epochs_M[i], method='pli', mode='multitaper', sfreq=sfreq, fmin=fmin, fmax=fmax,
        faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)
    list_con_M.append(con)

for i in range(len(list_epochs_S)):
    con = spectral_connectivity_epochs(
        list_epochs_S[i], method='pli', mode='multitaper', sfreq=sfreq, fmin=fmin, fmax=fmax,
        faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)
    list_con_S.append(con)

for i in range(len(list_epochs_M)):
    con = spectral_connectivity_epochs(
        list_epochs_O[i], method='pli', mode='multitaper', sfreq=sfreq, fmin=fmin, fmax=fmax,
        faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)
    list_con_O.append(con)

In [ ]:
'''

这里的思路是把每个被试的connectivity的data拿出来进行平均,然后作图.

只是简单地去看一下趋势 没有统计学意义.

'''


def average_connectivity_data(list_con, nodes):
    """
    Compute the average connectivity from a list of SpectralConnectivity objects.

    """
    # Create an empty list to store the connectivity data
    data_list = []

    # Iterate through the list of SpectralConnectivity objects and retrieve the data
    for con in list_con:
        data_list.append(con.get_data(output='dense')[:nodes, :nodes, 0])

    # Compute the average of the data arrays
    average_data = np.mean(data_list, axis=0)

    return average_data

average_data_M = average_connectivity_data(list_con_M,63)
average_data_S = average_connectivity_data(list_con_S,63)
average_data_O = average_connectivity_data(list_con_O,63)
plot_sensors_connectivity(list_epochs_M[0].info, average_data_O, cbar_label= 'O')
plot_sensors_connectivity(list_epochs_M[0].info, average_data_S, cbar_label= 'S')
plot_sensors_connectivity(list_epochs_M[0].info, average_data_M, cbar_label= 'M')



In [37]:

'''
另一种方法是 所有被试的所有epoch 合并到一起.
'''

epochs_M_all = mne.concatenate_epochs(list_epochs_M)
epochs_S_all = mne.concatenate_epochs(list_epochs_S)
epochs_O_all = mne.concatenate_epochs(list_epochs_O)

con_M_all = spectral_connectivity_epochs(
        epochs_M_all, method='pli', mode='multitaper', sfreq=sfreq, fmin=fmin, fmax=fmax,
        faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)
con_S_all = spectral_connectivity_epochs(
        epochs_S_all, method='pli', mode='multitaper', sfreq=sfreq, fmin=fmin, fmax=fmax,
        faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)
con_O_all = spectral_connectivity_epochs(
        epochs_O_all, method='pli', mode='multitaper', sfreq=sfreq, fmin=fmin, fmax=fmax,
        faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)

plot_sensors_connectivity(epochs_M_all.info, con_M_all.get_data(output='dense')[:63, :63, 0], cbar_label= 'M')
plot_sensors_connectivity(epochs_M_all.info, con_S_all.get_data(output='dense')[:63, :63, 0], cbar_label= 'S')
plot_sensors_connectivity(epochs_M_all.info, con_O_all.get_data(output='dense')[:63, :63, 0], cbar_label= 'O')

Not setting metadata
1095 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
1095 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
1095 matching events found
Applying baseline correction (mode: mean)
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 2016 connections
    using t=0.250s..0.800s for estimation (276 points)
    frequencies: 5.4Hz..7.2Hz (2 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoc

C:\Users\SilverBullet\AppData\Local\Temp\ipykernel_25460\389538732.py:9: RuntimeWarning: There were no Annotations stored in <Epochs |  1095 events (all good), -0.2 - 0.8 sec, baseline -0.2 – 0 sec, ~268.0 MB, data loaded,
 '102': 1095>, so metadata was not modified.
  con_M_all = spectral_connectivity_epochs(
C:\Users\SilverBullet\AppData\Local\Temp\ipykernel_25460\389538732.py:9: RuntimeWarning: fmin=4.000 Hz corresponds to 2.208 < 5 cycles based on the epoch length 0.552 sec, need at least 1.250 sec epochs or fmin=9.058. Spectrum estimate will be unreliable.
  con_M_all = spectral_connectivity_epochs(


    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    computing connectivity for epoch 35
    computing connectivity for epoch 36
    computing connectivity for epoch 37
    computing connectivity for epoch 38
    computing connectivity for epoch 39
    computing connectivity for epoch 40
    computing connectivity for epoch 41
    computing connectivity for epoch 42
    computing connectivity for epoch 43
    computing connectivity for epoch 44
    computing connectivity for epoch 45
    computing connectivity for epoch 46
    computing connectivity for epoch 47
    computing connectivity for epoch 48


C:\Users\SilverBullet\AppData\Local\Temp\ipykernel_25460\389538732.py:12: RuntimeWarning: There were no Annotations stored in <Epochs |  1095 events (all good), -0.2 - 0.8 sec, baseline -0.2 – 0 sec, ~268.0 MB, data loaded,
 '103': 1095>, so metadata was not modified.
  con_S_all = spectral_connectivity_epochs(
C:\Users\SilverBullet\AppData\Local\Temp\ipykernel_25460\389538732.py:12: RuntimeWarning: fmin=4.000 Hz corresponds to 2.208 < 5 cycles based on the epoch length 0.552 sec, need at least 1.250 sec epochs or fmin=9.058. Spectrum estimate will be unreliable.
  con_S_all = spectral_connectivity_epochs(


    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    computing connectivity for epoch 35
    computing connectivity for epoch 36
    computing connectivity for epoch 37
    computing connectivity for epoch 38
    computing connectivity for epoch 39
    computing connectivity for epoch 40
    computing connectivity for epoch 41
    computing connectivity for epoch 42
    computing connectivity for epoch 43
    computing connectivity for epoch 44
    computing connectivity for epoch 45
    computing connectivity for epoch 46
    computing connectivity for epoch 47
    computing connectivity for epoch 48
    computing connectivity for epoch 49
    computing connectivity for epoch 50
    computing connectivity for epoch 51
    computing connectivity for epoch 52
    computing connectivity for epoch 53
    computing connectivity for epoch 54
    computing connectivity for epoch 55
    computing connectivity for epoch 56


C:\Users\SilverBullet\AppData\Local\Temp\ipykernel_25460\389538732.py:15: RuntimeWarning: There were no Annotations stored in <Epochs |  1095 events (all good), -0.2 - 0.8 sec, baseline -0.2 – 0 sec, ~268.0 MB, data loaded,
 '101': 1095>, so metadata was not modified.
  con_O_all = spectral_connectivity_epochs(
C:\Users\SilverBullet\AppData\Local\Temp\ipykernel_25460\389538732.py:15: RuntimeWarning: fmin=4.000 Hz corresponds to 2.208 < 5 cycles based on the epoch length 0.552 sec, need at least 1.250 sec epochs or fmin=9.058. Spectrum estimate will be unreliable.
  con_O_all = spectral_connectivity_epochs(


    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    computing connectivity for epoch 35
    computing connectivity for epoch 36
    computing connectivity for epoch 37
    computing connectivity for epoch 38
    computing connectivity for epoch 39
    computing connectivity for epoch 40
    computing connectivity for epoch 41
    computing connectivity for epoch 42
    computing connectivity for epoch 43
    computing connectivity for epoch 44
    computing connectivity for epoch 45
    computing connectivity for epoch 46
    computing connectivity for epoch 47
    computing connectivity for epoch 48
    computing connectivity for epoch 49
    computing connectivity for epoch 50
    computing connectivity for epoch 51
    computing connectivity for epoch 52
    computing connectivity for epoch 53
    computing connectivity for epoch 54
    computing connectivity for epoch 55
